We depend on this sync_lib library that is one folder above

In [63]:
import sys
!pip install tabulate
sys.path.append('../')

You should consider upgrading via the '/Users/jeromel/anaconda2/envs/deep_work/bin/python -m pip install --upgrade pip' command.


In [12]:
from sync_lib import Dataset
import matplotlib.pylab as plt
import numpy as np
import glob
import pandas as pd

Path to relevant sync file

In [7]:
list_h5_files = glob.glob("*/*.h5")

In [8]:
list_h5_files

['det_change_long_no_licks/2021T143928.h5',
 'detection_of_change_with_camera/2021T15144.h5',
 'det_change_short_manual/2021T17922.h5',
 'GridGratingDrawing/2021T134531.h5',
 'keyboard_test_1/go_no_go_hardware_2021T155621.h5',
 'detection_of_change_without_camera/2021T151449.h5',
 'det_change_long_fake_lick_generator/2021T133340.h5',
 'PassiveViewing_run1/2021T135255.h5',
 'long_stim_test_1/go_no_go_hardware_2021T12950.h5']

In [64]:
rendering_mean = []
rendering_std = []
photodiode_driving_mean = []
photodiode_driving_std = []
photodiode_response_mean = []
photodiode_response_std = []
photodiode_dropped = []

index = []
for indiv_path in list_h5_files:
    local_folder = indiv_path.split('/')[0]
    dset = Dataset(indiv_path)
    if not('GridGratingDrawing' in local_folder):
        if 'detection_of_change_with_camera' in local_folder:
            local_folder = 'det_change_with_camera'
        if 'detection_of_change_without_camera' in local_folder:
            local_folder = 'det_change_without_camera'
        if 'PassiveViewing_run1' in local_folder:
            local_folder = 'passive_viewing_long'
        if 'long_stim_test_1' in local_folder:
            local_folder = 'go_no_go_' + local_folder 
        if 'keyboard_test_1' in local_folder:
            local_folder = 'go_no_go_' + local_folder 

        index.append(local_folder)

        # This is the stim rendering period
        times_bonsai_fast_sync = dset.get_rising_edges('vsync_stim', units='sec')
        rendering_mean.append(np.mean(np.diff(times_bonsai_fast_sync[10:-10])))
        rendering_std.append(np.std(np.diff(times_bonsai_fast_sync[10:-10])))

        # This is the driving signal behind the photodiode
        times_bonsai_driving_photodiode = dset.get_rising_edges(
            'stim_running', units='sec')
        photodiode_driving_mean.append(np.mean(np.diff(times_bonsai_driving_photodiode[10:-10])))
        photodiode_driving_std.append(np.std(np.diff(times_bonsai_driving_photodiode[10:-10])))

        # This is the measured photodiode signal
        times_photodiode = dset.get_rising_edges('stim_photodiode', units='sec')
        photodiode_response_mean.append(np.mean(np.diff(times_photodiode[10:-10])))
        
        period_photodiode = np.diff(times_photodiode[10:-10])

        photodiode_response_std.append(np.std(period_photodiode))
        
        delta_photodiode = np.abs(np.diff(period_photodiode)-np.mean(np.diff(period_photodiode)))
        photodiode_dropped.append(len(delta_photodiode[delta_photodiode>0.033]))

all_dict = {'Bonsai rendering period mean':rendering_mean, 'Bonsai rendering std': rendering_std, 'Photodiode driving period mean': photodiode_driving_mean, 'Photodiode driving period std':photodiode_driving_std, 'Photodiode actual period mean':photodiode_response_mean, 'Photodiode actual period std':photodiode_response_std, 'Photodiode dropped frames':photodiode_dropped}
df = pd.DataFrame(all_dict, columns = ['Bonsai rendering period mean'
    ,'Bonsai rendering std', 'Photodiode driving period mean', 'Photodiode driving period std', 'Photodiode actual period mean', 'Photodiode actual period std', 'Photodiode dropped frames'], index = index)

from pandas import DataFrame
from tabulate import tabulate

print(tabulate(df, tablefmt="pipe", headers="keys"))


|                                     |   Bonsai rendering period mean |   Bonsai rendering std |   Photodiode driving period mean |   Photodiode driving period std |   Photodiode actual period mean |   Photodiode actual period std |   Photodiode dropped frames |
|:------------------------------------|-------------------------------:|-----------------------:|---------------------------------:|--------------------------------:|--------------------------------:|-------------------------------:|----------------------------:|
| det_change_long_no_licks            |                      0.0333329 |            0.000253716 |                         0.999986 |                     0.000158512 |                        0.999986 |                    1.67488e-05 |                           0 |
| det_change_with_camera              |                      0.0333386 |            0.000942805 |                         0.999984 |                     6.55527e-05 |                        0.999984 |        